# GraphSAGE, GCN, GIN

---

In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab


In [3]:
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-geometric
!pip install ogb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 7.9 MB 2.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 3.5 MB 2.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 407 kB 4.9 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=078330b2b0a8db00d3667b5313118d4cd71ac3f7c8e7e34ca857727a4e0f45a5
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c

In [4]:
import torch
import os
os.chdir('/content/drive/MyDrive/data/aesthetics') 

import os.path as osp
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges
from data import get_data, get_data_split

In [5]:
path = ""
data = get_data(path)
data.edge_index=data.edge_index.long()
train_data, val_data, test_data = get_data_split(path)
train_data.edge_index=train_data.edge_index.long()
val_data.edge_index=val_data.edge_index.long()
test_data.edge_index=test_data.edge_index.long()
data = train_test_split_edges(data)

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [6]:
dota = get_data(path)
dota.edge_index=dota.edge_index.long()
dota.edge_index

tensor([[11, 11,  8, 16, 17,  5, 13, 42, 29,  4,  5,  8,  0, 13, 14, 21,  2, 28,
         12, 25, 14, 34, 11, 13, 14, 16, 29, 21, 11, 20, 16, 25,  4, 11,  4, 15,
         11, 10, 20, 13,  4,  7, 35, 17,  5, 13, 17, 18, 29, 16, 20, 31, 24, 11,
          4, 14, 10,  0,  6, 11, 22, 22, 11,  5, 15,  8,  2,  8, 34, 15,  4,  6,
         17,  5, 14, 17,  2, 29, 17,  8, 24, 16, 25, 13, 37,  8, 20, 28, 23, 41,
          4, 15, 21,  2, 12,  4, 39, 23, 31, 37, 11, 17,  6, 14, 39, 35, 36, 16,
         20, 20, 11,  5, 24,  2, 13, 12,  5, 11, 31, 25,  4,  5, 23, 20,  1,  4,
         36,  6,  7, 38, 23, 16, 41, 17, 21, 20, 25, 16, 14, 16, 15, 15, 24,  1,
         20,  2,  6, 14,  2, 34,  4, 24, 28, 23, 31, 31, 17, 15, 39,  7,  7, 28,
         14, 23,  8,  5, 23, 10, 22, 25, 11, 21, 36,  4, 15,  4, 22],
        [15, 20, 13, 18, 23, 13, 32, 43, 38, 17, 27, 29,  2, 35, 22, 28, 31, 33,
         23, 35, 20, 36, 14, 31, 15, 25, 37, 24, 23, 34, 42, 41, 23, 29, 31, 22,
         18, 27, 29, 20, 37, 27, 39, 40

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# If you use GPU, the device should be cuda
print('Device: {}'.format(device))

Device: cpu


---
----

# Other kind

----
---

In [17]:
import torch
import torch.nn.functional as F
from torch.nn import Sequential, Linear, BatchNorm1d, ReLU

from torch_geometric.utils import negative_sampling, to_dense_adj, dense_to_sparse
from torch_geometric.data import Data

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv, GINConv, GATConv

from ogb.linkproppred import PygLinkPropPredDataset, Evaluator
from torch_geometric.loader import DataLoader
from torch_sparse import SparseTensor
from torch_geometric.nn import Node2Vec

import pandas as pd
import shutil, os
import os.path as osp
import numpy as np

#from logger import Logger
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import random

In [37]:
class SAGE(torch.nn.Module):
  ''' Define GCN network. '''
  def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

  def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

  def forward(self, x, adj_t):
      # Execute conv -> relu -> dropout sequence
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x


class GCN(torch.nn.Module):
  ''' Define GCN network. '''
  def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels, cached=True))
        for _ in range(num_layers - 2):
            self.convs.append(
                GCNConv(hidden_channels, hidden_channels, cached=True))
        self.convs.append(GCNConv(hidden_channels, out_channels, cached=True))

        self.dropout = dropout

  def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

  def forward(self, x, adj_t):
        # Execute conv -> relu -> dropout sequence
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x


class GIN(torch.nn.Module):
  ''' Define graph isomorphic network. '''
  def __init__(self, in_channels, dim, out_channels, num_layers,dropout):
        super().__init__()
         # # Initialize 2 GINConv layers (following num_layers=2 in model-agnostic hyperparams as specified in the blogpost)
        self.conv1 = GINConv(
            Sequential(Linear(in_channels, dim), BatchNorm1d(dim), ReLU(),
                       Linear(dim, dim), ReLU())) # GINConv takes a neural network as input

        self.conv2 = GINConv(
            Sequential(Linear(dim, dim), BatchNorm1d(dim), ReLU(),
                       Linear(dim, dim), ReLU()))
        
        self.dropout = dropout
  
  def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()

  def forward(self, x, adj_t):
       # Execute conv -> relu -> dropout sequence
        x = self.conv1(x, adj_t)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, adj_t)
        return x

# class in order to predict whether a link exists between two nodes using 
# their embeddings, x_i and x_j
class LinkPredictor(torch.nn.Module):
    ''' Neural network which predicts whether a link (interaction) exists between 2 nodes i,j 
    given their embeddings x_i, x_j. 
    '''
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(LinkPredictor, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()

    def forward(self, x_i, x_j):
        x = x_i * x_j # hadamard product
        for lin in self.lins[:-1]: # linear layer -> relu -> dropout
            x = lin(x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.sigmoid(x) # sigmoid activation outputs probability that a given edge exists for all node pairs

class DotProductLinkPredictor(torch.nn.Module):
    def __init__(self):
        super(DotProductLinkPredictor, self).__init__()

    def forward(self, x_i, x_j):
        out = (x_i*x_j).sum(-1)
        return torch.sigmoid(out)
    
    def reset_parameters(self):
      pass

In [42]:
gnn_args = { # define GNN hyperparams
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'hidden_size': 25,
    'input_size': data.x.shape[1],
    'dropout': 0.5,
    'epochs': 100,
    'weight_decay': 1e-5,
    'lr': 0.005,
    'attn_size': 32,
    'num_layers':2,
    'log_steps':100,
    'eval_steps':5,
    'runs':10,
    'batch_size': 64*1024,

}

In [20]:
adj_t = SparseTensor(row=dota.edge_index[0], col=dota.edge_index[1],
                   sparse_sizes=(data.num_nodes, data.num_nodes))

In [25]:
def train(model, predictor, adj_t, data, optimizer, batch_size):

    embedding= torch.nn.Embedding(data.x.shape[0],data.x.shape[1]).to(device)

    embedding.weight.data.copy_(data.x)
    x=embedding.weight

    row, col = data.train_pos_edge_index
    edge_index = torch.stack([col, row], dim=0) # data.train_pos_edge_index mit vertauschten Elementen

    model.train()
    predictor.train()

    pos_train_edge = data.train_pos_edge_index.T.to(x.device) # [node1, node 1a],....

    total_loss = total_examples = 0
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size,
                           shuffle=True):
        optimizer.zero_grad()

        h = model(x, adj_t)

        edge = pos_train_edge[perm].t()

        # computes the loss for positive edges
        pos_out = predictor(h[edge[0]], h[edge[1]])
        pos_loss = -torch.log(pos_out + 1e-15).mean()

        # samples negative edges from the graph
        edge = negative_sampling(edge_index, num_nodes=x.size(0),
                                 num_neg_samples=perm.size(0), method='dense')

        # computes the loss for negative edges
        neg_out = predictor(h[edge[0]], h[edge[1]])
        neg_loss = -torch.log(1 - neg_out + 1e-15).mean()

        loss = pos_loss + neg_loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(x, 1.0)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(predictor.parameters(), 1.0)

        optimizer.step()

        num_examples = pos_out.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples

    return total_loss / total_examples


@torch.no_grad()
def test(model, predictor, adj_t, data,  batch_size):
    model.eval()
    predictor.eval()

    embedding= torch.nn.Embedding(data.x.shape[0],data.x.shape[1]).to(device)

    embedding.weight.data.copy_(data.x) 
    x=embedding.weight

    h = model(x, adj_t)
    
    pos_train_edge = data.train_pos_edge_index.T.to(x.device)
    pos_valid_edge = data.val_pos_edge_index.T.to(x.device)
    neg_valid_edge = data.val_neg_edge_index.T.to(x.device)
    pos_test_edge = data.test_pos_edge_index.T.to(x.device)
    neg_test_edge = data.test_neg_edge_index.T.to(x.device)

    # store what the link predictor outputs for each positive and negative 
    # edge in order to compute the hits@K
    pos_train_preds = []
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size):
        edge = pos_train_edge[perm].t()
        pos_train_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_train_pred = torch.cat(pos_train_preds, dim=0)

    pos_valid_preds = []
    for perm in DataLoader(range(pos_valid_edge.size(0)), batch_size):
        edge = pos_valid_edge[perm].t()
        pos_valid_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_valid_pred = torch.cat(pos_valid_preds, dim=0)

    neg_valid_preds = []
    for perm in DataLoader(range(neg_valid_edge.size(0)), batch_size):
        edge = neg_valid_edge[perm].t()
        neg_valid_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    neg_valid_pred = torch.cat(neg_valid_preds, dim=0)

    pos_test_preds = []
    for perm in DataLoader(range(pos_test_edge.size(0)), batch_size):
        edge = pos_test_edge[perm].t()
        pos_test_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_test_pred = torch.cat(pos_test_preds, dim=0)

    neg_test_preds = []
    for perm in DataLoader(range(neg_test_edge.size(0)), batch_size):
        edge = neg_test_edge[perm].t()
        neg_test_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    neg_test_pred = torch.cat(neg_test_preds, dim=0)

    # compute the hits@K for training, validation, and test

    def compute_loss(pos_score, neg_score):  # computes the loss based on binary cross entropy
        scores = torch.cat([pos_score, neg_score])
        labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
        return F.binary_cross_entropy_with_logits(scores, labels)

    def compute_auc(pos_score, neg_score):  # computes AUC (Area-Under-Curve) score
        scores = torch.cat([pos_score, neg_score]).numpy()
        labels = torch.cat(
            [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
        return roc_auc_score(labels, scores)
    results = {}
    train_hits=compute_auc(pos_train_pred, neg_test_pred)
    test_hits=compute_auc(pos_test_pred, neg_test_pred)
    valid_hits=compute_auc(pos_valid_pred, neg_valid_pred)
    results[f'Hm'] = (train_hits, valid_hits, test_hits)

  

    return results

In [31]:
def train_model(model, data, gnn_args, predictor, model_name): 
  ''' 
  Train specified GNN model. Model and embeddings should be initialized. 
  Save model after every run. 
  '''
  train_hits_arr, val_hits_arr, test_hits_arr = [], [], []
  
  embedding= torch.nn.Embedding(data.x.shape[0],data.x.shape[1]).to(device)

  embedding.weight.data.copy_(data.x) 

  emb=embedding

  split_edge=dota.edge_index.T

  
  for run in range(20):
    max_valhits, train_hits_run, test_hits_run = float('-inf'), 0, 0
    
    
    model.reset_parameters()
    predictor.reset_parameters()
    optimizer = torch.optim.Adam(
        list(model.parameters()) + list(emb.parameters()) +
        list(predictor.parameters()), lr=gnn_args['lr'])

    for epoch in range(1, 1 + gnn_args['epochs']):
        loss = train(model, predictor,  adj_t, data,optimizer, gnn_args['batch_size'])
        if epoch % gnn_args['eval_steps'] == 0:
            results = test(model, predictor, adj_t, data, gnn_args['batch_size'])
            

            if epoch % gnn_args['log_steps'] == 0:
                for key, result in results.items():
                    train_hits, valid_hits, test_hits = result
                    print(key)
                    print(f'Run: {run + 1:02d}, '
                          f'Epoch: {epoch:02d}, '
                          f'Loss: {loss:.4f}, '
                          f'Train: {100 * train_hits:.2f}%, '
                          f'Valid: {100 * valid_hits:.2f}%, '
                          f'Test: {100 * test_hits:.2f}%')
                print('---')
    
            # check val-hits@20
            train_hits, valid_hits, test_hits = results['Hm']
            if valid_hits >= max_valhits: # if validhits20 is higher than max, save ckpt               
              max_valhits = valid_hits
              train_hits_run = train_hits
              test_hits_run = test_hits
              # Save model checkpoint for current run.
              model_path = f"training_outputs/{model_name}.pt"
              emb_path = f'training_outputs/{model_name}_init_emb.pt'
              #save_model_ckpt(model, emb, optimizer, predictor, loss, emb_path, model_path)
    train_hits_arr.append(train_hits_run)
    test_hits_arr.append(test_hits_run)
    val_hits_arr.append(max_valhits)


  # Print overall stats arrays for best model based on val hits@20
  print("Val_roc: ", val_hits_arr)
  print("Test_roc: ", test_hits_arr)
  print("Train_roc: ", train_hits_arr)

  # Print best model stats (based on val hits@20)
  val_max = max(val_hits_arr)
  print("Best model val: ", max(val_hits_arr))
  max_idx = val_hits_arr.index(val_max)
  print('Best model test: ', test_hits_arr[max_idx])
  print('Best model train: ', val_hits_arr[max_idx])

  # convert to numpy array
  val_hits_arr = np.array(val_hits_arr)
  test_hits_arr = np.array(test_hits_arr)
  train_hits_arr = np.array(train_hits_arr)

  # Print average stats + variance
  print(f"Average best train: {np.mean(train_hits_arr)}; var: {np.var(train_hits_arr)}")
  print(f"Average best val: {np.mean(val_hits_arr)}; var: {np.var(val_hits_arr)}")
  print(f"Average best test: {np.mean(test_hits_arr)}; var: {np.var(test_hits_arr)}")

In [32]:
from sklearn.metrics import roc_auc_score

model_name = 'sage'
sage_model = SAGE(gnn_args['input_size'], gnn_args['hidden_size'], gnn_args['hidden_size'], 
              gnn_args['num_layers'], gnn_args['dropout']).to(device)  
predictor = LinkPredictor(gnn_args['hidden_size'], gnn_args['hidden_size'], 1,
                          gnn_args['num_layers'], gnn_args['dropout']).to(device)

train_model(sage_model, data, gnn_args, predictor, 'sage')

Hm
Run: 01, Epoch: 100, Loss: 0.8700, Train: 92.61%, Valid: 96.88%, Test: 87.20%
---
Hm
Run: 02, Epoch: 100, Loss: 1.0911, Train: 88.39%, Valid: 89.06%, Test: 88.93%
---
Hm
Run: 03, Epoch: 100, Loss: 0.8419, Train: 96.01%, Valid: 92.19%, Test: 92.39%
---
Hm
Run: 04, Epoch: 100, Loss: 0.8745, Train: 91.25%, Valid: 89.06%, Test: 87.89%
---
Hm
Run: 05, Epoch: 100, Loss: 0.8381, Train: 91.83%, Valid: 89.06%, Test: 89.27%
---
Hm
Run: 06, Epoch: 100, Loss: 0.7787, Train: 95.78%, Valid: 93.75%, Test: 93.08%
---
Hm
Run: 07, Epoch: 100, Loss: 0.8194, Train: 91.29%, Valid: 93.75%, Test: 88.24%
---
Hm
Run: 08, Epoch: 100, Loss: 0.8266, Train: 89.51%, Valid: 89.06%, Test: 86.85%
---
Hm
Run: 09, Epoch: 100, Loss: 0.8208, Train: 96.98%, Valid: 93.75%, Test: 93.77%
---
Hm
Run: 10, Epoch: 100, Loss: 0.9636, Train: 95.70%, Valid: 90.62%, Test: 94.81%
---
Hm
Run: 11, Epoch: 100, Loss: 0.6547, Train: 94.93%, Valid: 87.50%, Test: 92.04%
---
Hm
Run: 12, Epoch: 100, Loss: 0.9598, Train: 93.27%, Valid: 93.75

In [43]:
model_name = 'gcn'

predictor = LinkPredictor(gnn_args['hidden_size'], gnn_args['hidden_size'], 1,
                          gnn_args['num_layers'], gnn_args['dropout']).to(device)

gcn_model = GCN(gnn_args['input_size'], gnn_args['hidden_size'], gnn_args['hidden_size'], 
             gnn_args['num_layers'], gnn_args['dropout']).to(device)

train_model(gcn_model, data, gnn_args, predictor, 'gcn')

Hm
Run: 01, Epoch: 100, Loss: 0.7996, Train: 90.91%, Valid: 85.94%, Test: 89.27%
---
Hm
Run: 02, Epoch: 100, Loss: 0.8697, Train: 87.85%, Valid: 84.38%, Test: 87.89%
---
Hm
Run: 03, Epoch: 100, Loss: 0.8448, Train: 93.54%, Valid: 87.50%, Test: 96.19%
---
Hm
Run: 04, Epoch: 100, Loss: 0.8732, Train: 92.69%, Valid: 89.06%, Test: 93.08%
---
Hm
Run: 05, Epoch: 100, Loss: 0.8160, Train: 91.68%, Valid: 89.06%, Test: 94.46%
---
Hm
Run: 06, Epoch: 100, Loss: 0.8430, Train: 91.37%, Valid: 85.94%, Test: 92.73%
---
Hm
Run: 07, Epoch: 100, Loss: 0.9096, Train: 95.59%, Valid: 90.62%, Test: 97.23%
---
Hm
Run: 08, Epoch: 100, Loss: 1.0162, Train: 92.65%, Valid: 87.50%, Test: 92.73%
---
Hm
Run: 09, Epoch: 100, Loss: 0.9320, Train: 92.03%, Valid: 84.38%, Test: 94.12%
---
Hm
Run: 10, Epoch: 100, Loss: 0.9591, Train: 90.60%, Valid: 90.62%, Test: 93.43%
---
Hm
Run: 11, Epoch: 100, Loss: 0.9435, Train: 90.63%, Valid: 84.38%, Test: 92.39%
---
Hm
Run: 12, Epoch: 100, Loss: 0.9445, Train: 90.79%, Valid: 87.50

In [45]:
model_name='gin'
predictor = LinkPredictor(gnn_args['hidden_size'], gnn_args['hidden_size'], 1,
                          gnn_args['num_layers'], gnn_args['dropout']).to(device)

gin_model = GIN(gnn_args['input_size'], gnn_args['hidden_size'], gnn_args['hidden_size'], 
             gnn_args['num_layers'], gnn_args['dropout']).to(device)

train_model(gin_model, data, gnn_args, predictor, 'gin')

Hm
Run: 01, Epoch: 100, Loss: 0.9222, Train: 97.33%, Valid: 84.38%, Test: 94.12%
---
Hm
Run: 02, Epoch: 100, Loss: 0.9593, Train: 92.53%, Valid: 93.75%, Test: 90.31%
---
Hm
Run: 03, Epoch: 100, Loss: 0.9103, Train: 90.33%, Valid: 89.06%, Test: 86.85%
---
Hm
Run: 04, Epoch: 100, Loss: 0.8708, Train: 91.18%, Valid: 95.31%, Test: 87.89%
---
Hm
Run: 05, Epoch: 100, Loss: 0.9182, Train: 90.17%, Valid: 87.50%, Test: 86.85%
---
Hm
Run: 06, Epoch: 100, Loss: 0.9103, Train: 92.30%, Valid: 90.62%, Test: 92.39%
---
Hm
Run: 07, Epoch: 100, Loss: 0.8983, Train: 89.40%, Valid: 92.19%, Test: 87.89%
---
Hm
Run: 08, Epoch: 100, Loss: 0.8991, Train: 91.33%, Valid: 87.50%, Test: 91.00%
---
Hm
Run: 09, Epoch: 100, Loss: 0.8601, Train: 94.70%, Valid: 90.62%, Test: 89.97%
---
Hm
Run: 10, Epoch: 100, Loss: 0.9659, Train: 89.59%, Valid: 87.50%, Test: 88.24%
---
Hm
Run: 11, Epoch: 100, Loss: 0.8917, Train: 88.97%, Valid: 98.44%, Test: 83.74%
---
Hm
Run: 12, Epoch: 100, Loss: 1.0852, Train: 89.36%, Valid: 87.50